## 어텐션 메커니즘을 이용한 번역기 구현

Seq2Seq을 사용한 번역기에서 어텐션 메커니즘을 추가하여 기계 번역기 개선

### 데이터 로드 및 전처리

In [1]:
import re
import os
import unicodedata
import urllib3
import zipfile
import shutil
import numpy as np
import pandas as pd
import torch
from collections import Counter
from tqdm import tqdm
from torch.utils.data import DataLoader, TensorDataset

In [2]:
# 사용할 총 데이터의 양
num_samples = 230000

In [3]:
def unicode_to_ascii(s):
    # 프랑스어 악센트(accent) 삭제
    # 예시 : 'déjà diné' -> deja dine
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

In [4]:
def preprocess_sentence(sent):
    # 악센트 삭제 함수 호출
    sent = unicode_to_ascii(sent.lower())

    # 단어와 구두점 사이에 공백을 만듭니다.
    # Ex) "he is a boy." => "he is a boy ."
    sent = re.sub(r"([?.!,¿])", r" \1", sent)

    # (a-z, A-Z, ".", "?", "!", ",") 이들을 제외하고는 전부 공백으로 변환합니다.
    sent = re.sub(r"[^a-zA-Z!.?]+", r" ", sent)

    # 다수 개의 공백을 하나의 공백으로 치환
    sent = re.sub(r"\s+", " ", sent)
    return sent

In [5]:
def load_preprocessed_data():
    encoder_input, decoder_input, decoder_target = [], [], []

    with open("fra.txt", "r") as lines:
        for i, line in enumerate(lines):
            # source 데이터와 target 데이터 분리
            src_line, tar_line, _ = line.strip().split('\t')

            # source 데이터 전처리
            src_line = [w for w in preprocess_sentence(src_line).split()]

            # target 데이터 전처리
            tar_line = preprocess_sentence(tar_line)
            tar_line_in = [w for w in ("<sos> " + tar_line).split()]
            tar_line_out = [w for w in (tar_line + " <eos>").split()]

            encoder_input.append(src_line)
            decoder_input.append(tar_line_in)
            decoder_target.append(tar_line_out)

            if i == num_samples - 1:
                break

    return encoder_input, decoder_input, decoder_target

In [6]:
# 전처리 테스트
en_sent = u"Have you had dinner?"
fr_sent = u"Avez-vous déjà diné?"

print('전처리 전 영어 문장 :', en_sent)
print('전처리 후 영어 문장 :',preprocess_sentence(en_sent))
print('전처리 전 프랑스어 문장 :', fr_sent)
print('전처리 후 프랑스어 문장 :', preprocess_sentence(fr_sent))

전처리 전 영어 문장 : Have you had dinner?
전처리 후 영어 문장 : have you had dinner ?
전처리 전 프랑스어 문장 : Avez-vous déjà diné?
전처리 후 프랑스어 문장 : avez vous deja dine ?


In [7]:
# 전처리 수행
# 훈련 시 사용할 디코더의 입력 시퀀스와 출력 시퀀스를 따로 분리하여 저장
# 입력 시퀀스에는 시작을 의미하는 토큰인 <sos>를 추가
# 출력 시퀀스에는 종료를 의미하는 토큰인 <eos>를 추가
sents_en_in, sents_fra_in, sents_fra_out = load_preprocessed_data()
print('인코더의 입력 :',sents_en_in[:5])
print('디코더의 입력 :',sents_fra_in[:5])
print('디코더의 레이블 :',sents_fra_out[:5])

인코더의 입력 : [['go', '.'], ['go', '.'], ['go', '.'], ['go', '.'], ['hi', '.']]
디코더의 입력 : [['<sos>', 'va', '!'], ['<sos>', 'marche', '.'], ['<sos>', 'en', 'route', '!'], ['<sos>', 'bouge', '!'], ['<sos>', 'salut', '!']]
디코더의 레이블 : [['va', '!', '<eos>'], ['marche', '.', '<eos>'], ['en', 'route', '!', '<eos>'], ['bouge', '!', '<eos>'], ['salut', '!', '<eos>']]


훈련 과정에서는 이전 시점의 디코더 셀의 출력을 현재 시점의 디코더 셀의 입력으로 넣어주지 않고, 

이전 시점의 실제값을 현재 시점의 디코더 셀의 입력값으로 하는 방법을 사용

그 이유는 이전 시점의 디코더 셀의 예측이 틀렸는데 이를 현재 시점의 디코더 셀의 입력으로 사용하면,

현재 시점의 디코더 셀의 예측도 잘못될 가능성이 높고 이는 연쇄 작용으로 디코더 전체의 예측을 어렵게 함

이런 상황이 반복되면 훈련 시간이 느려짐

이 상황을 원하지 않는다면 이전 시점의 디코더 셀의 예측값 대신 실제값을 현재 시점의 디코더 셀의 입력으로 사용하는 방법을 사용할 수 있음

이와 같이 RNN의 모든 시점에 대해서 이전 시점의 예측값 대신 실제값을 입력으로 주는 방법을 교사 강요라고 함

In [8]:
# 단어 집합(Vocabulary) 생성 함수
# 패딩 토큰을 위한 <PAD> 토큰은 0번, OOV에 대응하기 위한 <UNK> 토큰은 1번에 할당
# 빈도수가 가장 높은 단어는 정수가 2번, 빈도수가 두번 째로 많은 단어는 정수 3번이 할당
def build_vocab(sents):
    word_list = []

    for sent in sents:
        for word in sent:
            word_list.append(word)

    # 각 단어별 등장 빈도를 계산하여 등장 빈도가 높은 순서로 정렬
    word_counts = Counter(word_list)
    vocab = sorted(word_counts, key=word_counts.get, reverse=True)

    word_to_index = {}
    word_to_index['<PAD>'] = 0
    word_to_index['<UNK>'] = 1

    # 등장 빈도가 높은 단어일수록 낮은 정수를 부여
    for index, word in enumerate(vocab) :
        word_to_index[word] = index + 2

    return word_to_index

In [9]:
# 구현 방식에 따라서는 하나의 단어 집합으로 만들어도 상관없음
# 영어 단어 집합
src_vocab = build_vocab(sents_en_in)
# 프랑스어 단어 집합
tar_vocab = build_vocab(sents_fra_in + sents_fra_out)

src_vocab_size = len(src_vocab)
tar_vocab_size = len(tar_vocab)
print("영어 단어 집합의 크기 : {:d}, 프랑스어 단어 집합의 크기 : {:d}".format(src_vocab_size, tar_vocab_size))

영어 단어 집합의 크기 : 15516, 프랑스어 단어 집합의 크기 : 25011


In [10]:
index_to_src = {v: k for k, v in src_vocab.items()}
index_to_tar = {v: k for k, v in tar_vocab.items()}

In [11]:
# 정수로부터 단어를 얻는 딕셔너리 생성
def texts_to_sequences(sents, word_to_index):
    encoded_X_data = []
    for sent in tqdm(sents):
        index_sequences = []
        for word in sent:
            try:
                index_sequences.append(word_to_index[word])
            except KeyError:
                index_sequences.append(word_to_index['<UNK>'])
        encoded_X_data.append(index_sequences)
    return encoded_X_data

In [12]:
encoder_input = texts_to_sequences(sents_en_in, src_vocab)
decoder_input = texts_to_sequences(sents_fra_in, tar_vocab)
decoder_target = texts_to_sequences(sents_fra_out, tar_vocab)

100%|██████████| 230000/230000 [00:00<00:00, 295360.23it/s]


In [13]:
# 상위 샘플에 대해서 정수 인코딩 전, 후 문장 출력
# 인코더 입력이므로 <sos>나 <eos>가 없음
for i, (item1, item2) in zip(range(10), zip(sents_en_in, encoder_input)):
    print(f"Index: {i}, 정수 인코딩 전: {item1}, 정수 인코딩 후: {item2}")

Index: 0, 정수 인코딩 전: ['go', '.'], 정수 인코딩 후: [50, 2]
Index: 1, 정수 인코딩 전: ['go', '.'], 정수 인코딩 후: [50, 2]
Index: 2, 정수 인코딩 전: ['go', '.'], 정수 인코딩 후: [50, 2]
Index: 3, 정수 인코딩 전: ['go', '.'], 정수 인코딩 후: [50, 2]
Index: 4, 정수 인코딩 전: ['hi', '.'], 정수 인코딩 후: [2488, 2]
Index: 5, 정수 인코딩 전: ['hi', '.'], 정수 인코딩 후: [2488, 2]
Index: 6, 정수 인코딩 전: ['run', '!'], 정수 인코딩 후: [406, 116]
Index: 7, 정수 인코딩 전: ['run', '!'], 정수 인코딩 후: [406, 116]
Index: 8, 정수 인코딩 전: ['run', '!'], 정수 인코딩 후: [406, 116]
Index: 9, 정수 인코딩 전: ['run', '!'], 정수 인코딩 후: [406, 116]


In [14]:
# 패딩 함수
def pad_sequences(sentences, max_len=None):
    # 최대 길이 값이 주어지지 않을 경우 데이터 내 최대 길이로 패딩
    if max_len is None:
        max_len = max([len(sentence) for sentence in sentences])

    features = np.zeros((len(sentences), max_len), dtype=int)
    for index, sentence in enumerate(sentences):
        if len(sentence) != 0:
            features[index, :len(sentence)] = np.array(sentence)[:max_len]
    return features

In [15]:
encoder_input = pad_sequences(encoder_input)
decoder_input = pad_sequences(decoder_input)
decoder_target = pad_sequences(decoder_target)

In [16]:
print('인코더의 입력의 크기(shape) :',encoder_input.shape)
print('디코더의 입력의 크기(shape) :',decoder_input.shape)
print('디코더의 레이블의 크기(shape) :',decoder_target.shape)

인코더의 입력의 크기(shape) : (230000, 20)
디코더의 입력의 크기(shape) : (230000, 28)
디코더의 레이블의 크기(shape) : (230000, 28)


In [17]:
# 테스트 데이터를 분리하기 전 데이터를 섞음
indices = np.arange(encoder_input.shape[0])
np.random.shuffle(indices)
print('랜덤 시퀀스 :',indices)

랜덤 시퀀스 : [ 56600 182004 155052 ...   5579 207840 138745]


In [18]:
encoder_input = encoder_input[indices]
decoder_input = decoder_input[indices]
decoder_target = decoder_target[indices]

In [19]:
print([index_to_src[word] for word in encoder_input[10000]])
print([index_to_tar[word] for word in decoder_input[10000]])
print([index_to_tar[word] for word in decoder_target[10000]])

['he', 'deliberately', 'broke', 'the', 'glass', '.', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']
['<sos>', 'il', 'a', 'deliberement', 'brise', 'la', 'vitre', '.', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']
['il', 'a', 'deliberement', 'brise', 'la', 'vitre', '.', '<eos>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']


In [20]:
n_of_val = int(33000*0.1)
print('검증 데이터의 개수 :',n_of_val)

검증 데이터의 개수 : 3300


In [21]:
encoder_input_train = encoder_input[:-n_of_val]
decoder_input_train = decoder_input[:-n_of_val]
decoder_target_train = decoder_target[:-n_of_val]

encoder_input_test = encoder_input[-n_of_val:]
decoder_input_test = decoder_input[-n_of_val:]
decoder_target_test = decoder_target[-n_of_val:]

In [22]:
print('훈련 source 데이터의 크기 :',encoder_input_train.shape)
print('훈련 target 데이터의 크기 :',decoder_input_train.shape)
print('훈련 target 레이블의 크기 :',decoder_target_train.shape)
print('테스트 source 데이터의 크기 :',encoder_input_test.shape)
print('테스트 target 데이터의 크기 :',decoder_input_test.shape)
print('테스트 target 레이블의 크기 :',decoder_target_test.shape)

훈련 source 데이터의 크기 : (226700, 20)
훈련 target 데이터의 크기 : (226700, 28)
훈련 target 레이블의 크기 : (226700, 28)
테스트 source 데이터의 크기 : (3300, 20)
테스트 target 데이터의 크기 : (3300, 28)
테스트 target 레이블의 크기 : (3300, 28)


### 번역기 모델링

In [23]:
import torch
import torch.nn as nn
import torch.optim as optim

In [24]:
embedding_dim = 256
hidden_units = 256

In [25]:
# Encoder 클래스 Seq2Seq 기계 번역기와 동일
class Encoder(nn.Module):
    def __init__(self, src_vocab_size, embedding_dim, hidden_units):
        super(Encoder, self).__init__()
        self.embedding = nn.Embedding(src_vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_units, batch_first=True)

    def forward(self, x):
        # x.shape == (batch_size, seq_len, embedding_dim)
        x = self.embedding(x)
        # hidden.shape == (1, batch_size, hidden_units), cell.shape == (1, batch_size, hidden_units)
        outputs, (hidden, cell) = self.lstm(x)
        return outputs, hidden, cell

In [26]:
# Decoder 클래스, Seq2Seq 기계 번역기와 완전히 달라짐, 어텐션 메커니즘 자체가 디코더 단에서 시작되기 때문
# 디코더의 은닉 상태와 인코더의 모든 시점의 은닉 상태간의 내적(dot product)를 통해서 어텐션 스코어(attention_scores)를 계산
# 어텐션 스코어를 소프트맥스(softmax) 함수를 통과시켜 어텐션 가중치(attention_weights)를 계산
# 어텐션 가중치는 어텐션 메커니즘에서 Value에 해당하는 인코더의 모든 시점의 은닉 상태와 다시 각각 곱하고 모두 더해 컨텍스트 벡터(context_vector)를 계산
class Decoder(nn.Module):
    def __init__(self, tar_vocab_size, embedding_dim, hidden_units):
        super(Decoder, self).__init__()
        self.embedding = nn.Embedding(tar_vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim + hidden_units, hidden_units, batch_first=True)
        self.fc = nn.Linear(hidden_units, tar_vocab_size)
        self.softmax = nn.Softmax(dim=2)

    def forward(self, x, encoder_outputs, hidden, cell):
        x = self.embedding(x)

        # Dot product attention
        # attention_scores.shape: (batch_size, source_seq_len, 1)
        attention_scores = torch.bmm(encoder_outputs, hidden.transpose(0, 1).transpose(1, 2))

        # attention_weights.shape: (batch_size, source_seq_len, 1)
        attention_weights = self.softmax(attention_scores)

        # context_vector.shape: (batch_size, 1, hidden_units)
        context_vector = torch.bmm(attention_weights.transpose(1, 2), encoder_outputs)

        # Repeat context_vector to match seq_len
        # context_vector_repeated.shape: (batch_size, target_seq_len, hidden_units)
        seq_len = x.shape[1]
        context_vector_repeated = context_vector.repeat(1, seq_len, 1)

        # Concatenate context vector and embedded input
        # x.shape: (batch_size, target_seq_len, embedding_dim + hidden_units)
        x = torch.cat((x, context_vector_repeated), dim=2)

        # output.shape: (batch_size, target_seq_len, hidden_units)
        # hidden.shape: (1, batch_size, hidden_units)
        # cell.shape: (1, batch_size, hidden_units)
        output, (hidden, cell) = self.lstm(x, (hidden, cell))

        # output.shape: (batch_size, target_seq_len, tar_vocab_size)
        output = self.fc(output)

        return output, hidden, cell

In [27]:
# Seq2Seq 클래스, Seq2Seq 기계 번역기와 동일
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, src, trg):
        encoder_outputs, hidden, cell = self.encoder(src)
        output, _, _ = self.decoder(trg, encoder_outputs, hidden, cell)
        return output

In [28]:
encoder = Encoder(src_vocab_size, embedding_dim, hidden_units)
decoder = Decoder(tar_vocab_size, embedding_dim, hidden_units)
model = Seq2Seq(encoder, decoder)

In [29]:
loss_function = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adam(model.parameters())

In [30]:
print(model)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(15516, 256, padding_idx=0)
    (lstm): LSTM(256, 256, batch_first=True)
  )
  (decoder): Decoder(
    (embedding): Embedding(25011, 256, padding_idx=0)
    (lstm): LSTM(512, 256, batch_first=True)
    (fc): Linear(in_features=256, out_features=25011, bias=True)
    (softmax): Softmax(dim=2)
  )
)


In [31]:
def evaluation(model, dataloader, loss_function, device):
    model.eval()
    total_loss = 0.0
    total_correct = 0
    total_count = 0

    with torch.no_grad():
        for encoder_inputs, decoder_inputs, decoder_targets in dataloader:
            encoder_inputs = encoder_inputs.to(device)
            decoder_inputs = decoder_inputs.to(device)
            decoder_targets = decoder_targets.to(device)

            # 순방향 전파
            # outputs.shape == (batch_size, seq_len, tar_vocab_size)
            outputs = model(encoder_inputs, decoder_inputs)

            # 손실 계산
            # outputs.view(-1, outputs.size(-1))의 shape는 (batch_size * seq_len, tar_vocab_size)
            # decoder_targets.view(-1)의 shape는 (batch_size * seq_len)
            loss = loss_function(outputs.view(-1, outputs.size(-1)), decoder_targets.view(-1))
            total_loss += loss.item()

            # 정확도 계산 (패딩 토큰 제외)
            mask = decoder_targets != 0
            total_correct += ((outputs.argmax(dim=-1) == decoder_targets) * mask).sum().item()
            total_count += mask.sum().item()

    return total_loss / len(dataloader), total_correct / total_count

In [32]:
encoder_input_train_tensor = torch.tensor(encoder_input_train, dtype=torch.long)
decoder_input_train_tensor = torch.tensor(decoder_input_train, dtype=torch.long)
decoder_target_train_tensor = torch.tensor(decoder_target_train, dtype=torch.long)

encoder_input_test_tensor = torch.tensor(encoder_input_test, dtype=torch.long)
decoder_input_test_tensor = torch.tensor(decoder_input_test, dtype=torch.long)
decoder_target_test_tensor = torch.tensor(decoder_target_test, dtype=torch.long)

In [33]:
# 데이터셋 및 데이터로더 생성
batch_size = 128

train_dataset = TensorDataset(encoder_input_train_tensor, decoder_input_train_tensor, decoder_target_train_tensor)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

valid_dataset = TensorDataset(encoder_input_test_tensor, decoder_input_test_tensor, decoder_target_test_tensor)
valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

In [34]:
# 학습 설정
num_epochs = 20
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(15516, 256, padding_idx=0)
    (lstm): LSTM(256, 256, batch_first=True)
  )
  (decoder): Decoder(
    (embedding): Embedding(25011, 256, padding_idx=0)
    (lstm): LSTM(512, 256, batch_first=True)
    (fc): Linear(in_features=256, out_features=25011, bias=True)
    (softmax): Softmax(dim=2)
  )
)

In [35]:
# Training loop
best_val_loss = float('inf')

for epoch in range(num_epochs):
    # 훈련 모드
    model.train()

    for encoder_inputs, decoder_inputs, decoder_targets in train_dataloader:
        encoder_inputs = encoder_inputs.to(device)
        decoder_inputs = decoder_inputs.to(device)
        decoder_targets = decoder_targets.to(device)

        # 기울기 초기화
        optimizer.zero_grad()

        # 순방향 전파
        # outputs.shape == (batch_size, seq_len, tar_vocab_size)
        outputs = model(encoder_inputs, decoder_inputs)

        # 손실 계산 및 역방향 전파
        # outputs.view(-1, outputs.size(-1))의 shape는 (batch_size * seq_len, tar_vocab_size)
        # decoder_targets.view(-1)의 shape는 (batch_size * seq_len)
        loss = loss_function(outputs.view(-1, outputs.size(-1)), decoder_targets.view(-1))
        loss.backward()

        # 가중치 업데이트
        optimizer.step()

    train_loss, train_acc = evaluation(model, train_dataloader, loss_function, device)
    valid_loss, valid_acc = evaluation(model, valid_dataloader, loss_function, device)

    print(f'Epoch: {epoch+1}/{num_epochs} | Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | Valid Loss: {valid_loss:.4f} | Valid Acc: {valid_acc:.4f}')

    # 검증 손실이 최소일 때 체크포인트 저장
    if valid_loss < best_val_loss:
        print(f'Validation loss improved from {best_val_loss:.4f} to {valid_loss:.4f}. 체크포인트를 저장합니다.')
        best_val_loss = valid_loss
        torch.save(model.state_dict(), 'best_model_checkpoint.pth')

Epoch: 1/20 | Train Loss: 2.0988 | Train Acc: 0.6004 | Valid Loss: 2.1996 | Valid Acc: 0.5902
Validation loss improved from inf to 2.1996. 체크포인트를 저장합니다.
Epoch: 2/20 | Train Loss: 1.4554 | Train Acc: 0.6880 | Valid Loss: 1.7006 | Valid Acc: 0.6586
Validation loss improved from 2.1996 to 1.7006. 체크포인트를 저장합니다.
Epoch: 3/20 | Train Loss: 1.1230 | Train Acc: 0.7390 | Valid Loss: 1.4847 | Valid Acc: 0.6921
Validation loss improved from 1.7006 to 1.4847. 체크포인트를 저장합니다.
Epoch: 4/20 | Train Loss: 0.9138 | Train Acc: 0.7738 | Valid Loss: 1.3823 | Valid Acc: 0.7051
Validation loss improved from 1.4847 to 1.3823. 체크포인트를 저장합니다.
Epoch: 5/20 | Train Loss: 0.7707 | Train Acc: 0.8023 | Valid Loss: 1.3279 | Valid Acc: 0.7192
Validation loss improved from 1.3823 to 1.3279. 체크포인트를 저장합니다.
Epoch: 6/20 | Train Loss: 0.6731 | Train Acc: 0.8236 | Valid Loss: 1.2998 | Valid Acc: 0.7245
Validation loss improved from 1.3279 to 1.2998. 체크포인트를 저장합니다.
Epoch: 7/20 | Train Loss: 0.6014 | Train Acc: 0.8385 | Valid Loss: 

In [36]:
# 모델 로드
model.load_state_dict(torch.load('best_model_checkpoint.pth'))
model.to(device)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(15516, 256, padding_idx=0)
    (lstm): LSTM(256, 256, batch_first=True)
  )
  (decoder): Decoder(
    (embedding): Embedding(25011, 256, padding_idx=0)
    (lstm): LSTM(512, 256, batch_first=True)
    (fc): Linear(in_features=256, out_features=25011, bias=True)
    (softmax): Softmax(dim=2)
  )
)

In [37]:
# 검증 데이터에 대한 정확도와 손실 계산
val_loss, val_accuracy = evaluation(model, valid_dataloader, loss_function, device)

print(f'Best model validation loss: {val_loss:.4f}')
print(f'Best model validation accuracy: {val_accuracy:.4f}')

Best model validation loss: 1.2929
Best model validation accuracy: 0.7318


In [38]:
print(tar_vocab['<sos>'])
print(tar_vocab['<eos>'])

3
4


### seq2seq 기계 번역기 동작

seq2seq는 훈련 과정(교사 강요)과 테스트 과정에서의 동작 방식이 다름

테스트 과정을 위해 모델을 다시 설계해주어야 함, 특히 디코더를 수정해야 함

전체적인 번역 단계는 다음과 같음
  1. 번역하고자 하는 입력 문장이 인코더로 입력되어 인코더의 마지막 시점의 은닉 상태와 셀 상태를 얻음
  2. 인코더의 은닉 상태와 셀 상태, 그리고 토큰 \<sos\>를 디코더로 보냄
  3. 디코더가 토큰 \<eos\>가 나올 때까지 다음 단어를 예측하는 행동을 반복

In [39]:
index_to_src = {v: k for k, v in src_vocab.items()}
index_to_tar = {v: k for k, v in tar_vocab.items()}

In [40]:
# 원문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq_to_src(input_seq):
    sentence = ''
    for encoded_word in input_seq:
        if(encoded_word != 0):
            sentence = sentence + index_to_src[encoded_word] + ' '
    return sentence

In [41]:
# 번역문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq_to_tar(input_seq):
    sentence = ''
    for encoded_word in input_seq:
        if(encoded_word != 0 and encoded_word != tar_vocab['<sos>'] and encoded_word != tar_vocab['<eos>']):
            sentence = sentence + index_to_tar[encoded_word] + ' '
    return sentence

In [42]:
print(encoder_input_test[25])
print(decoder_input_test[25])
print(decoder_target_test[25])

[  10   12 4680    2    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0]
[   3   18   10 8677    2    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0]
[  18   10 8677    2    4    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0]


In [43]:
def decode_sequence(input_seq, model, src_vocab_size, tar_vocab_size, max_output_len, int_to_src_token, int_to_tar_token):
    encoder_inputs = torch.tensor(input_seq, dtype=torch.long).unsqueeze(0).to(device)

    # 인코더의 초기 상태 설정
    encoder_outputs, hidden, cell = model.encoder(encoder_inputs)

    # 시작 토큰 <sos>을 디코더의 첫 입력으로 설정
    # unsqueeze(0)는 배치 차원을 추가하기 위함.
    decoder_input = torch.tensor([3], dtype=torch.long).unsqueeze(0).to(device)

    decoded_tokens = []

    # for문을 도는 것 == 디코더의 각 시점
    for _ in range(max_output_len):
        output, hidden, cell = model.decoder(decoder_input, encoder_outputs, hidden, cell)

        # 소프트맥스 회귀를 수행. 예측 단어의 인덱스
        output_token = output.argmax(dim=-1).item()

        # 종료 토큰 <eos>
        if output_token == 4:
            break

        # 각 시점의 단어(정수)는 decoded_tokens에 누적하였다가 최종 번역 시퀀스로 리턴합니다.
        decoded_tokens.append(output_token)

        # 현재 시점의 예측. 다음 시점의 입력으로 사용된다.
        decoder_input = torch.tensor([output_token], dtype=torch.long).unsqueeze(0).to(device)

    return ' '.join(int_to_tar_token[token] for token in decoded_tokens)

In [44]:
for seq_index in [3, 50, 100, 300, 1000, 1500, 3000]:
    input_seq = encoder_input_train[seq_index]
    translated_text = decode_sequence(input_seq, model, src_vocab_size, tar_vocab_size, 20, index_to_src, index_to_tar)

    print("입력문장 :",seq_to_src(encoder_input_train[seq_index]))
    print("정답문장 :",seq_to_tar(decoder_input_train[seq_index]))
    print("번역문장 :",translated_text)
    print("-"*50)

입력문장 : tom wants to work . 
정답문장 : tom veut travailler . 
번역문장 : tom veut travailler .
--------------------------------------------------
입력문장 : i couldn t do that even if i wanted to . 
정답문장 : je ne pourrais pas faire cela meme si je le voulais . 
번역문장 : je ne pourrais pas faire ca si je voulais .
--------------------------------------------------
입력문장 : our only daughter died of cancer . 
정답문장 : notre fille unique est decedee d un cancer . 
번역문장 : notre fille unique est morte d un cancer .
--------------------------------------------------
입력문장 : i know how busy you ve been . 
정답문장 : je sais combien tu as ete occupee . 
번역문장 : je sais combien vous avez ete occupe .
--------------------------------------------------
입력문장 : how old is your house ? 
정답문장 : quel age a votre maison ? 
번역문장 : quel age a ta maison ?
--------------------------------------------------
입력문장 : all three were killed . 
정답문장 : tous les trois furent tues . 
번역문장 : toutes les trois furent tuees .
------------------

In [45]:
for seq_index in [3, 50, 100, 300, 1001, 1500, 3000]:
    input_seq = encoder_input_test[seq_index]
    translated_text = decode_sequence(input_seq, model, src_vocab_size, tar_vocab_size, 20, index_to_src, index_to_tar)

    print("입력문장 :",seq_to_src(encoder_input_test[seq_index]))
    print("정답문장 :",seq_to_tar(decoder_input_test[seq_index]))
    print("번역문장 :",translated_text)
    print("-"*50)

입력문장 : ask him whether she is at home or not . 
정답문장 : demande lui si elle est a la maison ou pas . 
번역문장 : a la maison lui elle est elle donc elle est a la ou pas .
--------------------------------------------------
입력문장 : tom should learn french . 
정답문장 : tom devrait apprendre le francais . 
번역문장 : tom devrait apprendre le francais .
--------------------------------------------------
입력문장 : i plead ignorance . 
정답문장 : j invoque l ignorance . 
번역문장 : je plaide l ignorance .
--------------------------------------------------
입력문장 : you don t look so busy . 
정답문장 : tu ne sembles pas si occupe . 
번역문장 : vous n avez pas l air si occupe .
--------------------------------------------------
입력문장 : you re after me . 
정답문장 : tu es apres moi . 
번역문장 : vous etes apres moi .
--------------------------------------------------
입력문장 : you re very busy . 
정답문장 : vous etes tres occupe . 
번역문장 : vous etes tres occupes .
--------------------------------------------------
입력문장 : how much is your commissi